# Forensics - VHD pyViewer

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__<br>
__OS Supported:__<br>
    -  Linux
__Platforms Supported:__<br>
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to analyze file system on newly created DSVM using Azure Forensics libraries.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Load .NET libraries 
2. Explore VHD 
3. File Management

## 1. Load .NET libraries

In [ ]:
import clr
clr.AddReference("System")
clr.AddReference("System.Collections")
clr.AddReference("System.Runtime")
clr.AddReference("Microsoft.Azure.Storage.Common")
clr.AddReference("Microsoft.Azure.Storage.Blob")
clr.AddReference("Microsoft.Azure.KeyVault.Core")
clr.AddReference("Newtonsoft.Json")
clr.AddReference("Microsoft.Azure.CIS.DiskLib")
clr.AddReference("Microsoft.Azure.CIS.DiskLib.Ntfs")
clr.AddReference("Microsoft.Azure.CIS.DiskLib.Vhd")
clr.AddReference("Microsoft.Azure.CIS.DiskLib.Vhd.Accessors")
clr.AddReference("PyHelper")

import sys
import clr
import os
import ipywidgets as widgets
import System
from System import *
from System.Collections.Generic import *
from Newtonsoft.Json import *
from Microsoft.Azure.CIS.DiskLib import *
from Microsoft.Azure.CIS.DiskLib.Ntfs import *
from Microsoft.Azure.CIS.DiskLib.Vhd import *
from Microsoft.Azure.CIS.DiskLib.Vhd.Accessors import *
from MftExportUtilityLib import *

In [ ]:
class MftHelper(object):
    p1 = 'Partition1'
    p2 = 'Partition2'
    p3 = 'Partition3'
    p4 = 'Partition4'
    
    def __init__(self, segment_start, sas_url, partition = None):
        self.uri = Uri(sas_url)
        r = Reader(PageBlobReader.Create(self.uri))
        self.mbr = r.GetMasterBootRecord()
        self.root = None
        self.segment = segment_start
        if partition is not None:
            if partition == self.p1:
                part = self.mbr.Partition1
            elif partition == self.p2:
                part = self.mbr.Partition2
            elif partition == self.p3:
                part = self.mbr.Partition3
            elif partition == self.p4:
                part = self.mbr.Partition4            
            self.scan_partition(part)

    def get_partitions(self):
        return [{self.p1 : hex(self.mbr.Partition1.PartitionType)},
                {self.p2 : hex(self.mbr.Partition2.PartitionType)},
                {self.p3 : hex(self.mbr.Partition3.PartitionType)},
                {self.p4 : hex(self.mbr.Partition4.PartitionType)}]
     
    def scan_partition(self, partition):
        ntfs_accessor = PageBlobReader.Create(self.uri, partition.LBAFirstSector *512)
        self.ntfs = NtfsData(ntfs_accessor)
        
        i = 0
        for mft_entry in self.ntfs.Mft:
            if i < int(self.segment,16): 
                i += 1
                continue
            self.root = mft_entry
            break

class FileHelper(object):
    def download_file(file_segment, ntfs):
        return MftExportUtil.DownloadFile(ntfs, int(file_segment, 16))[0]
    
    def display_file(file_path):
        with open(file_path, 'r', encoding="utf8", errors='ignore') as f:
            contents = f.read()
            print(contents)

            
class MftViewer(object):
    def __init__(self, mft_helper):
        self.root = mft_helper.root
        
    def display(self):
        box_folders = widgets.VBox()
        box_files = widgets.VBox()
        box_commander = widgets.HBox()
        self.update(box_commander, box_folders, box_files)
        return box_commander
    
    def update(self, box_commander, box_folders, box_files):
        self.load_box(box_folders, 1)
        self.load_box(box_files, 0)
        box_commander.children = [box_folders, box_files]
        
    def load_box(self, box, mode):
    #    buttons = []
        dataTable = []
        
        for node in self.root.Children:
            if mode == 0:
                if node.IsDirectory: continue
            else:
                if not node.IsDirectory: continue
            if node.FileNameData.Flags == 2: continue
            
      #      caption = node.Name
      #      button = widgets.Button(description=caption, background_color='#d0d0ff')
      #      buttons.append(button)
            dataTable.append([node.Name, hex(node.IndexEntry.FileReference.SegmentNumberLowPart), hex(node.IndexEntry.FileReference.SegmentNumberHighPart), hex(node.IndexEntry.FileReference.SequenceNumber)])

        import pandas as pd
        df = pd.DataFrame(dataTable, columns=["Name", "SegmentLow", "SegmentHigh", "SeqNum"])
        
        style = '''<style> .df th { text-align:center; font-size:large; padding-left:10px; padding-right:10px}  tbody tr:nth-child(even) { background-color: #f2f2f2; } td { padding-left: 10px; padding-right: 5px; }</style>'''
        box.children = tuple([widgets.HTML(style + df.to_html(classes="df"))])

## 2. Explorer VHD

1. Input SAS URL and Select Partition

In [ ]:
root_segment_low = "0x5"
sas_url = input('SAS Url:')

In [ ]:
# select an partition to continue
import ipywidgets as widgets
from IPython.display import display
mft_helper = MftHelper("0x5", sas_url)
partitions = mft_helper.get_partitions()
selected_partition = widgets.Dropdown(options=partitions, value=partitions[0],description='Partition:')
display(selected_partition)

2. Display Root Level Folders and Files

In [ ]:
# Display First level directories
#sas_url = input('SAS Url:')
mft_helper = MftHelper(root_segment_low, sas_url, next(iter(selected_partition.value.keys())))
MftViewer(mft_helper).display()

3. Randomly Access Directory and File

In [ ]:
# Random access to Directory / File, just replace the first parameter with selected SegmentLow
mft_helper = MftHelper("0xd33", sas_url, next(iter(selected_partition.value.keys())))
MftViewer(mft_helper).display()

## 3. File Management

In [ ]:
# File download 
# SYSTEM
#downloaded_file = FileHelper.download_file("0x17609", mft_helper.ntfs)
# SOFTWARE
downloaded_file = FileHelper.download_file("0x17609", mft_helper.ntfs)

In [ ]:
FileHelper.display_file(downloaded_file)